# 🐞 Autogen Assignment 5: Debugging and Logging Agent Interactions
This notebook demonstrates how to enable logging, simulate a failure case, analyze logs, and apply a fix.

In [5]:
# 🛠️ Step 1: Install required packages
!pip install pyautogen openai --quiet

In [6]:
# 🔐 Step 2: Set API key and disable Docker
import os
os.environ['OPENAI_API_KEY'] = ''
os.environ['AUTOGEN_USE_DOCKER'] = 'False'

In [7]:
# ✅ Step 3: Import Autogen and set up logging
import autogen
from autogen import AssistantAgent
import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [8]:
# 🧠 Step 4: Create a basic agent
config = {
    "llm_config": {
        "model": "gpt-3.5-turbo",
        "api_key": os.environ["OPENAI_API_KEY"]
    }
}

agent = AssistantAgent(
    name="DebugTutor",
    system_message="You are a Python tutor. Explain concepts clearly. If no code is provided, ask for it.",
    llm_config=config["llm_config"]
)

In [18]:
# 🔍 Step 5: Simulate problematic interaction
message = {"role": "user", "content": """Explain this code:

# Supported operators
operator_list = ['+', '-', '*', '**', '/','%', '//']

# Display operator options
print('Welcome to the Python Calculator!')
print('Select an operator from the list below:')
print('Available operations: ' + ', '.join(operator_list))

# Get user input
while True:
    try:
        num1 = float(input('Enter first number: '))
        operator = input('Enter operator (+, -, *, **, %, //): ')
        num2 = float(input('Enter second number: '))

        # Handle division/modulo by zero
        if operator in ['/','%', '//'] and num2 == 0:
            print("Cannot divide or modulo by zero.")
            continue

        # Perform operation
        if operator == '+':
            result = num1 + num2
        elif operator == '-':
            result = num1 - num2
        elif operator == '*':
            result = num1 * num2
        elif operator == '**':
            result = num1 ** num2
        elif operator == '/':
            result = num1/num2
        elif operator == '%':
            result = num1 % num2
        elif operator == '//':
            result = num1 // num2
        else:
            result = "Invalid operator selected."

        
        
        print(f"{num1} {operator} {num2} = {result}")
        break

    except ValueError:
        print('Invalid input! Please enter numeric values.')

          """ }
response = agent.generate_reply(messages=[message])
print("🧪 Agent Response:\n", response)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-3eb64bbd-1c43-4210-86b9-4535ab5b0d13', 'json_data': {'messages': [{'content': 'You are a Python tutor. Explain concepts clearly. If no code is provided, ask for it.', 'role': 'system'}, {'role': 'user', 'content': 'Explain this code:\n\n# Supported operators\noperator_list = [\'+\', \'-\', \'*\', \'**\', \'/\',\'%\', \'//\']\n\n# Display operator options\nprint(\'Welcome to the Python Calculator!\')\nprint(\'Select an operator from the list below:\')\nprint(\'Available operations: \' + \', \'.join(operator_list))\n\n# Get user input\nwhile True:\n    try:\n        num1 = float(input(\'Enter first number: \'))\n        operator = input(\'Enter operator (+, -, *, **, %, //): \')\n        num2 = float(input(\'Enter second number: \'))\n\n        # Handle division/modulo by zero\n        if operator in [\'/\',\'%\', \'//\'] and num2 == 0:\n    

🧪 Agent Response:
 This code is a simple Python calculator program. Here is an explanation of the code:

1. The program starts by defining a list of supported operators: ['+', '-', '*', '**', '/', '%', '//'].

2. It then displays a welcome message and prompts the user to select an operator from the list of available operations.

3. The program enters a while loop to repeatedly ask the user for input until a valid input is provided.

4. Inside the loop, the program attempts to read the first number, operator, and the second number from the user. If the input is not a numeric value, it raises a ValueError and prompts the user to enter numeric values.

5. It also includes a check to prevent division by zero. If the operator is division ('/'), modulo ('%'), or floor division ('//') and the second number is 0, it informs the user that division or modulo by zero is not allowed and continues to the next iteration of the loop.

6. Depending on the operator selected by the user, the program per

In [10]:
# ✅ Step 6: Create SafeDebugAgent with fallback response
class SafeDebugAgent(AssistantAgent):
    def __init__(self, name="SafeDebugTutor", **kwargs):
        system_message = "You are a Python tutor. If the user asks to explain code but provides none, ask for it."
        super().__init__(name=name, system_message=system_message, **kwargs)

    def generate_reply(self, messages, **kwargs):
        last_msg = messages[-1]['content'].lower()
        if "explain this code" in last_msg and len(last_msg.strip()) <= 25:
            return "🤖 Could you please share the code snippet you'd like me to explain?"
        return super().generate_reply(messages, **kwargs)

safe_agent = SafeDebugAgent(name="SafeDebugTutor", llm_config=config["llm_config"])
fixed_response = safe_agent.generate_reply(messages=[message])
print("🔧 Fixed Agent Response:\n", fixed_response)

🔧 Fixed Agent Response:
 🤖 Could you please share the code snippet you'd like me to explain?
